In [1]:
import pandas as pd
from GameFeatures import GameFeatures
import warnings
warnings.filterwarnings("ignore")
import pickle

In [2]:
df = pd.read_csv('gamesDB.csv')

In [6]:
df.columns

Index(['data', 'time_casa', 'time_visitante', 'resultado', 'id_partida',
       'link'],
      dtype='object')

In [1]:
#df.drop(columns='Unnamed: 0', inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 6 columns):
data              4560 non-null object
time_casa         4560 non-null object
time_visitante    4560 non-null object
resultado         4560 non-null object
id_partida        4560 non-null object
link              4560 non-null object
dtypes: object(6)
memory usage: 213.8+ KB


In [9]:
df.data[0]

'2007-05-12 19:10:00'

In [10]:
df.data = pd.to_datetime(df.data)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 6 columns):
data              4560 non-null datetime64[ns]
time_casa         4560 non-null object
time_visitante    4560 non-null object
resultado         4560 non-null object
id_partida        4560 non-null object
link              4560 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 213.8+ KB


In [12]:
mask = df.data.dt.year > 2009

In [13]:
df[mask].head()

,data,time_casa,time_visitante,resultado,id_partida,link
1140,2010-05-08 19:30:00,Botafogo,Santos,3-3,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1141,2010-05-08 19:30:00,Atlético GO,Grêmio,0-0,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1142,2010-05-08 19:35:00,Palmeiras,Vitória,1-0,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1143,2010-05-09 17:00:00,Flamengo,São Paulo,1-1,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1144,2010-05-09 17:00:00,Atlético MG,Vasco,2-1,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...


In [14]:
links = df.link[1140:1150]

In [15]:
home_team = []
away_team = []
home_team_score = []
away_team_score = []
referee = []
stadium = []
city = []
datetime = []
game = []
eventList1st = []
eventList2nd = []
lineupList = []
sublineupList = []
statsHome = []
statsAway = []

In [16]:
i = 0
games_len = len(links)
for url in links:
    home_teamT, away_teamT, home_team_scoreT, away_team_scoreT, refereeT, stadiumT, cityT, datetimeT, gameT, eventList1stT, eventList2ndT, lineupListT, sublineupListT, statsHomeT, statsAwayT = GameFeatures(url)
    
    home_team.append(home_teamT) 
    away_team.append(away_teamT) 
    home_team_score.append(home_team_scoreT) 
    away_team_score.append(away_team_scoreT) 
    referee.append(refereeT) 
    stadium.append(stadiumT) 
    city.append(cityT) 
    datetime.append(datetimeT) 
    game.append(gameT) 
    eventList1st.append(eventList1stT) 
    eventList2nd.append(eventList2ndT) 
    lineupList.append(lineupListT) 
    sublineupList.append(sublineupListT) 
    statsHome.append(statsHomeT) 
    statsAway.append(statsAwayT)
    
    i+=1
    print('Game number: '+str(i)+'/'+str(games_len))

Game number: 1/10
Game number: 2/10
Game number: 3/10
Game number: 4/10
Game number: 5/10
Game number: 6/10
Game number: 7/10
Game number: 8/10
Game number: 9/10
Game number: 10/10


In [17]:
df = pd.DataFrame({'home_team':home_team, 
                    'away_team':away_team, 
                    'home_team_score':home_team_score, 
                    'away_team_score':away_team_score, 
                    'referee':referee, 
                    'stadium':stadium, 
                    'city':city, 
                    'datetime':datetime, 
                    'game':game, 
                    'eventList1st':eventList1st, 
                    'eventList2nd':eventList2nd, 
                    'lineupList':lineupList, 
                    'sublineupList':sublineupList, 
                    'statsHome':statsHome, 
                    'statsAway':statsAway})

In [18]:
df

,home_team,away_team,home_team_score,away_team_score,referee,stadium,city,datetime,game,eventList1st,eventList2nd,lineupList,sublineupList,statsHome,statsAway
0,Botafogo,Santos,3,3,Leonardo Gaciba Da Silva,Estádio Nilton Santos,Rio de Janeiro,08 maio 2010 - 19:30,1,"[{'time': '8'', 'kind': 'Goal', 'hometeam': Tr...","[{'time': '46'', 'kind': 'Substitute in', 'hom...","([(person-4802, Jefferson), (person-16989, Ale...","([(person-32310, Renan), (person-32998, Marcel...",{},{}
1,Atlético,Grêmio,0,0,Salvio Spinola Fagundes Filho,Estádio do Governo do Estado de Goiás (Serra D...,"Goiânia, Goiás",08 maio 2010 - 19:30,1,"[{'time': '5'', 'kind': 'Yellow card', 'homete...","[{'time': '61'', 'kind': 'Substitute in', 'hom...","([(person-8906, Edson), (person-128779, Ayrton...","([(person-71130, Vitor Ressurreição), (person-...",{},{}
2,Palmeiras,Vitória,1,0,Pericles Bassols Pegado Cortez,Estádio Palestra Itália,"São Paulo, São Paulo",08 maio 2010 - 19:35,1,"[{'time': '21'', 'kind': 'Yellow card', 'homet...","[{'time': '52'', 'kind': 'Yellow card', 'homet...","([(person-9277, Marcos), (person-9164, Vitor),...","([(person-65191, Deola), (person-32928, Mauríc...",{},{}
3,Flamengo,São,1,1,Leandro Pedro Vuaden,Estadio Jornalista Mário Filho (Maracanã),"Rio de Janeiro, Rio de Janeiro",09 maio 2010 - 17:00,1,"[{'time': '10'', 'kind': 'Yellow card', 'homet...","[{'time': '46'', 'kind': 'Substitute in', 'hom...","([(person-9061, Bruno), (person-74270, Everton...","([(person-9062, Marcelo Lomba), (person-10614,...",{},{}
4,Atlético,Vasco,2,1,Elmo Alves Resende Cunha,Estádio Governador Magalhães Pinto,"Belo Horizonte, Minas Gerais",09 maio 2010 - 17:00,1,"[{'time': '9'', 'kind': 'Goal', 'hometeam': Tr...","[{'time': '53'', 'kind': 'Yellow card', 'homet...","([(person-32707, Aranha), (person-8908, Coelho...","([(person-10377, F. Carini), (person-23726, Pe...",{},{}
5,Internacional,Cruzeiro,1,2,Wilson Luiz Seneme,Estádio José Pinheiro Borba (Beira-Rio),"Porto Alegre, Rio Grande do Sul",09 maio 2010 - 17:00,1,"[{'time': '4'', 'kind': 'Penalty goal', 'homet...","[{'time': '46'', 'kind': 'Substitute in', 'hom...","([(person-9781, Lauro), (person-68732, Arilton...","([(person-130381, William Lago), (person-11352...",{},{}
6,Corinthians,Atlético,2,1,Marcelo de Lima Henrique,Estádio Municipal Paulo Machado de Carvalho,"São Paulo, São Paulo",09 maio 2010 - 17:00,1,"[{'time': '40'', 'kind': 'Yellow card', 'homet...","[{'time': '46'', 'kind': 'Substitute in', 'hom...","([(person-12425, Felipe), (person-32539, Aless...","([(person-9029, Júlio César), (person-4481, Pa...",{},{}
7,Ceará,Fluminense,1,0,Paulo Cesar De Oliveira,Estádio Governador Plácido Aderaldo Castelo,"Fortaleza, Ceará",09 maio 2010 - 19:30,1,"[{'time': '33'', 'kind': 'Red card', 'hometeam...","[{'time': '46'', 'kind': 'Substitute in', 'hom...","([(person-9063, Diego), (person-71755, Anderso...","([(person-9218, Michel Alves), (person-111502,...",{},{}
8,Guarani,Goiás,1,0,Arilson Bispo da Anunciação,Estádio Brinco de Ouro da Princesa,"Campinas, São Paulo",09 maio 2010 - 19:30,1,"[{'time': '33'', 'kind': 'Yellow card', 'homet...","[{'time': '46'', 'kind': 'Substitute in', 'hom...","([(person-65294, Douglas), (person-32486, Aíls...","([(person-112215, Juliano), (person-67994, Fab...",{},{}
9,Avaí,Grêmio,6,1,Djalma José Beltrami Teixeira,Estádio Aderbal Ramos da Silva,"Florianópolis, Santa Catarina",09 maio 2010 - 19:30,1,"[{'time': '8'', 'kind': 'Yellow card', 'homete...","[{'time': '46'', 'kind': 'Substitute in', 'hom...","([(person-79755, Zé Carlos), (person-33478, Pa...","([(person-127151, Renan), (person-65359, Pará)...",{},{}
